In [187]:
import datetime
import porterStemmerModule as psm
import re
import sys
import os
import snappy

In [188]:
#queryFile = 'myQuery'
queryFile = 'query.txt'
resultFile = 'resultFile4'
indexFile = 'IndexFile4.idx'
dictionaryFile = 'IndexFile4.dict'
# resultFile = 'myResult'
# indexFile = 'MyTestIndex.idx'
# dictionaryFile = 'MyTestIndex.dict'
stopwordFile = ''
compressionALgo = '3'
if len(sys.argv) > 5:
    queryFile = sys.argv[1]
    resultFile = sys.argv[2]
    indexFile = sys.argv[3]
    dictionaryFile = sys.argv[4]

In [189]:
stopwordsList = []
docnos = []
termDictionary = {}
with open(dictionaryFile, 'r') as f:
    line = f.readline()
    stopwordsList = line[:-1].strip('][').split(', ')
    line = f.readline()
    docnos = line[:-1].strip('][').split(', ')
    docnos =[i.strip('\'\'').strip() for i in docnos]
    compressionALgo = f.readline()[:-1]
    #print(compressionALgo)
    
    #Fteching dictionary
    line = f.readline()[:-1]
    #print(line)
    line = line.split(' ')
    termDictionary[line[0]] = [line[1], '0']
    #print(line[0], termDictionary[line[0]])
    prevKey = line[0]
    while True:
        line = f.readline()[:-1]
        if not line:
            break
        line = line.split(' ')
        termDictionary[line[0]] = [line[1], '0']
        termDictionary[prevKey][1]= line[1] 
        prevKey = line[0]

In [191]:
print(termDictionary['z'])
print(docnos[:10])

['66231289', '66231323']
['AP880212-0001', 'AP880212-0002', 'AP880212-0003', 'AP880212-0004', 'AP880212-0005', 'AP880212-0006', 'AP880212-0007', 'AP880212-0008', 'AP880212-0009', 'AP880212-0010']


In [192]:
def bitstring_to_bytes(bstr):
    return int(bstr, 2).to_bytes((len(bstr) + 7) // 8, byteorder='big')
def bytes_to_bitstrings(li):
    ans = ''
    for i in [bin(i)[2:] for i in li]:
        if len(i)%8 != 0:
            ans = ans + '0'*(8  - len(i)%8)
        ans = ans + i
    return ans 

In [193]:
def gapDecoding(postingListEncoded):
    for i in range(1, len(postingListEncoded)):
        postingListEncoded[i] = postingListEncoded[i] + postingListEncoded[i-1]
    return postingListEncoded

In [194]:
#assuming its bytestream stream
def decompressC1(plist):
    postingList = []
    #print(len(plist))
    numberOfBytes = int(len(plist))
    binaryItem = ''
    docNo = 0
    toAddNumber = 0
    for byt in plist:
        #print('for',byt)
        if byt < 128:
            docNo = docNo*128 + byt
            #print('appending doc number to plist', docNo)
            postingList.append(docNo)
            docNo = 0
        else:
            byt -= 128
            docNo = docNo*128 + byt
        #print('docNo',bin(docNo), docNo)
    postingList = gapDecoding(postingList)
    return postingList

In [195]:
#Taking int list for compression and Returning boolean string after compression (need to convert to list of bytes)
#Taking bool string before decompression and Returning int list after compression (need to convrt list of bytes to bool str)
### IMPORTANT ----- ITS ENCODING 0 AND 1 AS 0 (SO DON'T USE 0 FOR DENOTING DOCUMENT ID OR USE +1 AND -1 FOR ALL)

class compressionC2:
    def __init__(self):
        self.postingList = ''
        self.compressedList = ''
    def l(self,x):
        return len(bin(x)[2:])
    def lsb(self,x):
        return bin(x)[3:]
    def u(self, x):
        return '1'*(x - 1) + '0'
    def compressItem(self, item):
        lx = self.l(item)
        llx = self.l(lx)
        return self.u(llx) + self.lsb(lx) + self.lsb(item)
    def compress(self, postingsList):
        postingsList = gapEncoding(postingsList)
        postingsList = [ i+2 for i in postingsList]
        compressedList = ''
        for item in postingsList:
            compressedList = compressedList + self.compressItem(item)
        return compressedList
    def decompressItem(self, compressedList):
        index = 0
        llx = 0
        lx = ''
        while compressedList[index] == '1':
            llx += 1
            index +=1
        llx+=1
        index+=1
        #print('llx:', llx, 'index: ', index)
        lx = int('1' + compressedList[index : index + llx - 1],2)
        #print('lx:', lx)
        index = index + llx - 1
        #print(compressedList[index: index + lx -1])
        x = int('1' + compressedList[index : index + lx - 1],2)
        #print('x',x)
        return x, index + lx -1
    def decompress(self, compressedList):
        postingsList = []
        index = 0
        while index < len(compressedList) and compressedList[index] == '0':
            index+=1
        while index < len(compressedList):
            item, bitsProcessed = self.decompressItem(compressedList[index : ])
            index += bitsProcessed
            postingsList.append(item)
        postingsList = [ i -2 for i in postingsList]
        postingsList = gapDecoding(postingsList)
        return postingsList

In [196]:
def decompressC3(encodedList):
    if len(encodedList) == 0:
        return None
    postingsList = snappy.decompress(encodedList, 'UTF-8')
    postingsList = postingsList.split(',')
    return gapDecoding(postingsList)

In [197]:
def findPostingListPos(query):
    global termDictionary
    if query in termDictionary:
        start , end = termDictionary[query]
        return int(start), int(end)
    else:
        return None, None
# depricated code -->
#     with open('DictFile','r') as file:
#         while True:
#             line = file.readline()
#             if not line:
#                 break
#             term, posStrat = line.split(' ')
#             if term == query:
#                 nextLine = file.readline()
#                 if nextLine:
#                     posEnd = nextLine.split(' ')[1]
#                 else:
#                     posEnd = '0'
#                 return int(posStrat), int(posEnd)
#     return None, None            

In [198]:
def decompress(byte_arr, algo):
    if algo =='0':
        list_string = bytearray(byte_arr).decode()
        return list_string.strip('][').split(', ')
    if algo == '1':
        return decompressC1(byte_arr)
    elif algo == '2':
        string = bytes_to_bitstrings(byte_arr)
        #print(string)
        c2 = compressionC2()
        return c2.decompress(string)
    elif algo == '3':
        return decompressC3(byte_arr)
    elif algo == '4':
        return None
    elif algo == '5':
        return None

In [199]:
def getPostingList(algo, start, end):
    byt_arr = None
    global indexFile
    
    with open(indexFile, 'rb') as file:
        file.seek(start)
        if end != 0:
            byt_arr = file.read(end - start)
        else:
            byt_arr = file.read()
    #print('posting list byte:', byt_arr)
    return decompress(byt_arr, algo)

In [200]:
def boolsearch(query, algo='0'):
    ps = psm.PorterStemmer()
    query = ps.stem(query, 0, len(query)-1)
    start, end = findPostingListPos(query)
    #print(start,end)
    if start == None:
        #print('Not Found')
        return None
    return getPostingList(algo,start, end)

In [201]:
def writeQuery(qid, lst, sim):
    with open(resultFile,'a') as f:
        for l in lst:
            f.write('Q%s %s %s\n'%(qid, docnos[int(l)-1], sim))

In [202]:
def searchFromFile():
    global compressionALgo
    global queryFile
    global resultFile
    if os.path.exists(resultFile):
        os.remove(resultFile)
    with open(queryFile, 'r') as file:
        lines = file.readlines()
        sim = 1.0
        for qno, line in zip(range(len(lines)),lines):
            queries = list(filter(None, re.split("[ ,.:;\"'()\[\]{}`]+",line[:-1])))
            lst_set = None
            first = True
            #if qno > 40:
            #    print('for Query', queries)
            for query in queries:
                res = boolsearch(query, compressionALgo)
                print('q', query, 'result', res[:10])
                if res == None:
                    lst_set = None
                    break
                if first:
                    lst_set = set(res)
                    first = False
                else:
                    lst_set = lst_set & set(res)
                #print(lst_set)
            if lst_set != set() and lst_set != None:
                writeQuery(qno, lst_set, sim)
    return True

In [203]:
# queryFile = 'myQuery'
# resultFile = 'myResult'
# queryFile = 'query.txt'
# resultFile = 'resultFile'
searchFromFile()

q Airbus result ['657', '3490657', '3223490657', '3043223490657', '16883043223490657', '61616883043223490657', '6361616883043223490657', '166361616883043223490657', '65166361616883043223490657', '44665166361616883043223490657']
q Subsidies result ['328', '176328', '722176328', '3966722176328', '23903966722176328', '36223903966722176328', '12536223903966722176328', '12512536223903966722176328', '13812512536223903966722176328', '64013812512536223903966722176328']
q South result ['1', '61', '261', '5261', '165261', '25165261', '525165261', '7525165261', '27525165261', '6227525165261']
q African result ['9', '109', '41109', '4841109', '1274841109', '491274841109', '20491274841109', '13020491274841109', '5513020491274841109', '185513020491274841109']
q Sanctions result ['1742', '4801742', '534801742', '833534801742', '994833534801742', '110994833534801742', '195110994833534801742', '1173195110994833534801742', '3551173195110994833534801742', '1413551173195110994833534801742']
q Leveraged re

IndexError: cannot fit 'int' into an index-sized integer

In [137]:
# set_11 = set(boolsearch('Iran', '0'))
# print(len(set_11))
# set12 = set(boolsearch('Contra', '0'))
# print(len(set12))
# set_13 = set(boolsearch('Affair', '0'))
# print(len(set_13))
# set_ = set_11 & set12
# set_ = set_ & set_13
# print(len(set_))
# print('Main set ',[docnos[int(s)-1] for s in set_])

5045
2710
2484
29
Main set  ['AP890505-0013', 'AP890125-0124', 'AP880311-0176', 'AP890504-0122', 'AP890506-0131', 'AP880317-0072', 'AP890201-0036', 'AP881025-0013', 'AP880719-0189', 'AP880327-0056', 'AP880225-0139', 'AP880430-0145', 'AP880720-0023', 'AP881116-0051', 'AP890203-0211', 'AP880222-0020', 'AP881011-0151', 'AP880311-0204', 'AP881022-0094', 'AP880312-0018', 'AP890328-0142', 'AP881009-0032', 'AP881129-0150', 'AP880317-0225', 'AP890504-0157', 'AP880328-0153', 'AP880312-0129', 'AP890126-0018', 'AP880914-0005']


In [186]:
print(boolsearch('Airbus','3'))

['657', '3490657', '3223490657', '3043223490657', '16883043223490657', '61616883043223490657', '6361616883043223490657', '166361616883043223490657', '65166361616883043223490657', '44665166361616883043223490657', '66544665166361616883043223490657', '64066544665166361616883043223490657', '190764066544665166361616883043223490657', '...']


52


[8019,
 2137991,
 2140969,
 2156201,
 2156490,
 2156682,
 2157359,
 2162905,
 2164017,
 2167709,
 2170541,
 5103309,
 5106402,
 5109900,
 7567544,
 7582350,
 7593115,
 7598201,
 7613146,
 7617429,
 7617562,
 7617607,
 7617608,
 7617795,
 7619020,
 7619131]

6
for 190
docNo 0b111110 62
for 83
appending doc number to plist 8019
docNo 0b0 0
for 129
docNo 0b1 1
for 130
docNo 0b10000010 130
for 128
docNo 0b100000100000000 16640
for 52
appending doc number to plist 2129972
docNo 0b0 0


[8019, 2137991]

16640